In [ ]:
mode = 0 # 0 for subscribers, 1 for vidviews
graph_options = ['subscribers', 'vidviews']
url = 'https://socialblade.com/youtube/user/getmovies/monthly'
graph = "graph-youtube-monthly-{}-container".format(graph_options[mode])

In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs
from urllib.request import Request, urlopen

def open_social_blade_url(url):
    rq = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    urlop = urlopen(rq)
    if (urlop.status == 200):
        raw = urlop.read()
        bspage = bs(raw, 'html.parser')
        print(url)
    else:
        print(urlop.status)    
    return bspage

In [ ]:
import re
import js2xml

def find_chart_data_node(bspage, graph):
    script = bspage.find_all('script', {'type': 'text/javascript'}, text=re.compile(graph))[0].contents[0]
    parsed = js2xml.parse(script)
    bsjs = bs(js2xml.pretty_print(parsed), 'xml')
    bschart = bsjs.find('string',  text=re.compile(graph)).parent
    xchart = bschart.find('property', {'name': 'data'}).find('array')
    
    return xchart

In [ ]:
def extract_datapoints_to_list2(xchart):
    return [[int(j['value']) for j in i.find_all('number')] for i in xchart.find_all('array')]
    

In [ ]:
import pandas as pd

def convert_dplist_to_df(dplist):
    df = pd.DataFrame(dplist, columns = ['tstp', 'value'])
    df['dtime']=pd.to_datetime(df['tstp'], unit='ms')
    df['url']=url
    df['graph']=graph
    df = df[['url','graph','dtime', 'value']]
    return df

In [ ]:
def ParseOne(url, graph):
    
    bspage = open_social_blade_url(url)
    xchart = find_chart_data_node(bspage, graph)
    dplist = extract_datapoints_to_list2(xchart)
    df = convert_dplist_to_df(dplist)    
    
    return(df)

In [ ]:
dfout = ParseOne(url, graph)
dfout

In [ ]:
dfout.to_excel('./{}-parsed.xlsx'.format(graph.split('-')[3]))